# Code for Publication "Improving Hypertension Case Identification Using Natural Language Processing and Machine Learning on Free-Text Electronic Medical Records"

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn import tree
import graphviz 
import xgboost as xgb

import re
import datetime
import pickle
import xml.etree.ElementTree as ET
from os import listdir
from itertools import islice

# Process documents with cTAKES and parse output 

## Process Documents with cTAKES

In [ ]:
#Specifications to pass to cTAKES

#Your UMLS credientials
username = "***Your UMLS Username***"
password = "***Your UMLS Password***"

#Path to cTAKES clinical pipeline ***REPLACE WITH YOUR PATH***
pipeline = "~/Resources/apache-ctakes-4.0.0/bin/runClinicalPipeline.sh"

#Folder containing the documents you want processed, to simplify linkage I used the chart number (RHRN) 
#as the document name: RHRN.txt
inFolder = " -i data/DischargeSummaries/Text"

#Folder to put the annotated documents in
outFolder = " --xmiOut data/DischargeSummaries/cTAKESoutput/"

#UMLS credentials
UMLScred = f" --user {username} --pass {password}!"

#Final Shell command to execute
cmd = pipeline + inFolder + outFolder + UMLScred

!$cmd

## Parse cTAKES output 

- create a dataframe where each row is a discharge summary
- each column a CUI
- each element is the number of times a given CUI appeared in the discharge summary, non-negated or uncertain and referring to the patient

In [ ]:
floc = "data/DischargeSummaries/cTAKESoutput/"
cDocs = listdir(floc)
boc = pd.DataFrame()

for count, doc in enumerate(cDocs):
    tree = ET.parse(floc + doc) 
    root = tree.getroot()
        
    RHRN = doc.split('.')[0]
    
#     boc.loc[count] = 0
#     boc['RHRN'].loc[count] =RHRN
    
    concepts = root.findall(".//*[@ontologyConceptArr]")
    concepts = [x.attrib for x in concepts]
    concepts = pd.DataFrame(concepts)
    concepts = concepts[(concepts['subject'] == 'patient') & (concepts['polarity'] == '1')& (concepts['uncertainty']=='0')]
    
    cuis = root.findall(".//*[@cui]")
    tmp = {'RHRN':RHRN}
    for cui in cuis:
        concept_id = cui.attrib['{http://www.omg.org/XMI}id']
        if (concepts['ontologyConceptArr'].str.contains(concept_id).sum()) > 0:
            ind = cui.attrib['cui']
            try:
                tmp[ind]+=1
            except:
                tmp[ind] = 1
            
        
    boc = boc.append(tmp,ignore_index=True)
    
    if (count % 50)==0:
        print(count)

# Merge cTAKES labels with DAD and Chart Review data

## Merge output with DAD

In [ ]:
dad = pd.read_csv('data/DAD.csv',low_memory=False)
dad['RHRN'] = dad['RHRN'].astype(str)
boc = boc.merge(dad,on='RHRN')

## Merge with Chart review

In [ ]:
cr = pd.read_excel('data/ChartRev.xlsx',sheet_name='FULLDATA')
boc = boc.merge(cr,on='RHRN')

### Remove uncertain hypertension cases 

In [ ]:
boc =boc.loc[boc['Hypertension present'] != 'Maybe']

### Modify labels so 1 = Hypertension present and 0 = Hypertension not present

In [ ]:
BoC['Hypertension present'] = (BoC['Hypertension present'] == 'Yes').astype(int)

# Create columns to stratify on 

## Create a column indicating whether the patient died in hospital 

In [ ]:
# True = patient died, False = patient survived
boc['died'] = (boc['Chart_Disp'] == 'Died')

## Create a column with patient ages

In [ ]:
boc['BIRTHDATE']= pd.to_datetime(boc['BIRTHDATE'],format='%Y%m%d')
boc['ADMITDATE']= pd.to_datetime(boc['ADMITDATE'],format='%Y%m%d')
boc['Age'] = (boc['ADMITDATE'] - boc['BIRTHDATE']).astype('timedelta64[Y]')

## Cases are considered surgical if the DOCSVC1 column in the DAD contains any of

- '30|34|37|32|35|36|31|1003'

In [ ]:
# Variable to search on when stratifying by surgical cases
surg = '30|34|37|32|35|36|31|1003'

# Analyze Data

## Define some helper functions

In [ ]:
#always round .5 up
import decimal
context = decimal.getcontext()
context.rounding = decimal.ROUND_HALF_UP

In [ ]:
def format_out(func):
    def standard_round(*args):
        out = [int(round(decimal.Decimal(x*100), 0)) for x in func(*args)]
        return f"{out[0]}({out[0]-out[1]}-{out[0]+out[1]})"
    return standard_round

@format_out
def precision(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    p = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return p, ci(p,sum(predicted))

@format_out
def recall(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    r = sum(np.where(predicted & actual,1,0))/sum(actual)
    return r, ci(r,sum(actual))

@format_out
def accuracy(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    a = sum(np.where(predicted==actual,1,0))/len(actual)
    return a, ci(a,len(actual))

@format_out
def specificity(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    s = sum(np.where(predicted & actual,1,0))/sum(actual)
    return s, ci(s,sum(actual))

@format_out
def NPV(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    n = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return n, ci(n,sum(predicted))

# 95% confidence intervals
def ci(p,N):
    return 1.96*(p*(1-p)/N)**0.5


def stats(predicted,actual):
    print("Sample Size = ", len(actual))
    print("Positive Cases = ",sum(actual))
    print("Cases Labeled Positive = ", sum(predicted))
#     print("Negative Cases = ", sum(1-actual))
#     print("Cases Labeled Negative = ", sum(1-predicted))    
    
    print("Recall = ",recall(predicted,actual))
    print("Specificity = ",specificity(predicted,actual))    
    print("Precision = ",precision(predicted,actual))
    print("NPV = ",NPV(predicted,actual))    
    print("Accuracy = ",accuracy(predicted,actual))    

## Create an ICD-10 case defintion column

In [ ]:
# Select out all columns with ICD diagnoses codes
cols = BoC.columns
codes = cols[cols.str.contains('dxcode',flags=re.IGNORECASE)]

# Create a column containing all diagnoes codes concatenated together
boc['AllCodes']=boc[codes].fillna("").apply(lambda row: " ".join([str(x) for x in row]),axis=1)

# Create a column that equal 1 if it satisfies the ICD-10 hypertension definition and 0 otherwise
BoC['ICD Hypt'] = (BoC['AllCodes'].str.contains('I10|I11|I12|I13|I15')).astype(int)

## Feature selection with XGBoost

### Select all CUIs as predictors

In [ ]:
cuis = boc.columns.str.contains('^C\d')

X = boc.loc[:,cuis]
Y = boc['Hypertension present']

### Specify parameters

In [ ]:
xgb_model = xgb.XGBClassifier()

parameters = {'nthread':[32], #when use hyperthread, xgboost may become slower
              'lambda':[0,0.5,1,2], #L2 regularization term on weights
              'alpha':[0,0.5,1,2], #L1 regularization term on weights 
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [3,5,6],
              'min_child_weight': [4,8,16],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [10,100,500,1000], #number of trees, change it to 1000 for better results
              'seed': [42]}

scoring = 'roc_auc'
kf = KFold(n_splits=5,shuffle=True,random_state=42)

### Create 5 Cross-Validated XGBoost models

In [ ]:
for fold, indices in enumerate(kf.split(X)):
    
    clf = GridSearchCV(xgb_model, parameters, n_jobs=6,
                       cv=10, 
                       scoring=scoring,
                       verbose=2, refit=True)

    X_train = X.iloc[indices[0]]
    Y_train = Y.iloc[indices[0]]
    clf.fit(X_train, Y_train)
    filename = f"models/Hypt-Model-FOLD-{fold}-CaseIdent-XGBoost-AllCUIs-SubjNegUncer.pkl"
    pickle.dump(clf, open(filename, 'wb'))

### Get stats for all folds

In [ ]:
folds = 5
kf = KFold(n_splits=folds,shuffle=True,random_state=42)
kf.get_n_splits(X)
rec = []
prec = []
for i in range(folds):
    train_ind, test_ind = next(islice(kf.split(X),i,i+1))
    X_samp = X.iloc[test_ind]
    Y_samp = Y.iloc[test_ind]
    
    filename = f'models/Hypt-Model-FOLD-{i}-CaseIdent-XGBoost-AllCUIs-NoICD11-SubjNegUncer.pkl'
    clf = pickle.load(open(filename, 'rb'))
    print(f'Fold {i} ---------------')
    pred = clf.predict(X_samp)
    rec.append(float(recall(pred,Y_samp)[0:2]))
    prec.append(float(precision(pred,Y_samp)[0:2]))
    stats(pred,Y_samp)

print('mean recall', sum(rec)/folds)
print('mean precision', sum(prec)/folds)

### Compare feature importances

In [ ]:
%matplotlib inline

In [ ]:
folds = 5

nf = 10

tfs = {}

for i in range(folds):
    filename = f'models/Hypt-Model-FOLD-{i}-CaseIdent-XGBoost-AllCUIs-SubjNegUncer.pkl'
    clf = pickle.load(open(filename, 'rb'))    
    
    tg = clf.best_estimator_.get_booster().get_score(importance_type= "gain")
    tg = pd.Series(tg)
    tg.sort_values(ascending=False,inplace=True)
    top_features = tg.iloc[:nf]
    # top_features

    tfs[i] = set(tg.index[:nf])

    pos = np.arange(top_features.shape[0])
    plt.subplot(folds, 1, i+1)
    plt.bar(pos,top_features)
#     plt.xticks(pos,top_features.index,rotation=-45)
    plt.ylabel("Feature Importance")
    plt.xlabel("Feature Rank")

### Select features that all appear in the top nf features

In [ ]:
agree = {}
for tf in tfs:
    try:
        agree = agree.intersection(tfs[tf])
    except:
        agree = tfs[tf]
        
agree

## Fit Decision tree using selected features

### Select features to use

In [ ]:
# Using the 4 features that appear in the top 10 features (by gain) of all 5 folds of the optimized XGBoost models
X = BoC[['C0020261', 'C0020538', 'C0025598', 'C0051696']]

In [ ]:
# Using the 3 features that appear in the top 10 features (by gain) of all 5 folds of the optimized XGBoost models that actually separate cases
X = BoC[['C0020261', 'C0020538', 'C0051696']]

In [ ]:
X.fillna(value=0,inplace=True)
Y = BoC['Hypertension present']

### Specify parameters

In [ ]:
parameters = {'criterion':['gini','entropy'],'max_depth':range(3,6),'min_samples_leaf':[3,5,10,20]}
scoring = 'roc_auc'

In [ ]:
rec = []
prec = []
for fold, indices in enumerate(kf.split(X)):
    
    clf = GridSearchCV(tree.DecisionTreeClassifier(splitter = 'best',random_state=42), 
                   parameters, cv = 10, 
                   n_jobs=4, scoring=scoring)

    X_train = X.iloc[indices[0]]
    Y_train = Y.iloc[indices[0]]
    clf.fit(X_train, Y_train)
    filename = f"models/Hypertension-Model-FOLD-{fold}-CaseIdent-CART-3CUIWhichSeparate-SubjNegUncer.pkl"
    pickle.dump(clf, open(filename, 'wb'))

    
    print(f'Fold {fold} ---------------')
    X_test = X.iloc[indices[1]]
    Y_test = Y.iloc[indices[1]]
    pred = clf.predict(X_test)
    rec.append(float(recall(pred,Y_test)[0:2]))
    prec.append(float(precision(pred,Y_test)[0:2]))
    stats(pred,Y_test)

print("*****AVERAGES*****")
print('mean recall', sum(rec)/folds)
print('mean precision', sum(prec)/folds)

### Visualize graph

In [ ]:
feature_names = ['Hydrochlorothiazide', 'Hypertensive disease','Amlodipine']
mdl = clf.best_estimator_
dot_data = tree.export_graphviz(mdl, feature_names=feature_names,out_file=None,filled=True, rounded = True,impurity=False)
graph = graphviz.

# Get stats for all models

## All Data

In [ ]:
X_samp = X
Y_samp = Y

### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'],BoC['Hypertension present'])

### Decision Tree

In [ ]:
pred = clf.predict(X_samp)
stats(pred,Y_samp)

### Combined

In [ ]:
pred = clf.predict(X_samp)
ICD = BoC['ICD Hypt']
combine = (pred | ICD.loc[X_samp.index]) 

In [ ]:
stats(combine,Y_samp)

## Stratified by Service (Surgical vs Not)

### Surgical

In [ ]:
# Select surgical patients
ll = BoC['DOCSVC1'].astype(str).str.contains(surg)

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[ll],BoC['Hypertension present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combine

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Non-Surgical

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[~ll],BoC['Hypertension present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combine

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])

## Stratified by Age (65 and older vs Under 65)

In [ ]:
# Select patients 65 and older
ll = BoC['Age'] > 64

### Older

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[ll],BoC['Hypertension present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combine

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Younger

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[~ll],BoC['Hypertension present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combine

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])

## Stratified by Mortality (Died vs Survived)

In [ ]:
# Select patients who died
ll = BoC['died']

### Died

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[ll],BoC['Hypertension present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combined

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Survived

#### ICD-10 Algorithm

In [ ]:
stats(BoC['ICD Hypt'].loc[~ll],BoC['Hypertension present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combined

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])